<a href="https://colab.research.google.com/github/YuvalNirkin/fsgan/blob/master/inference/face_swapping.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## FSGAN Face Swapping Demo
Fill out [this form](https://docs.google.com/forms/d/e/1FAIpQLScyyNWoFvyaxxfyaPLnCIAxXgdxLEMwR9Sayjh3JpWseuYlOA/viewform?usp=sf_link),
and after receiving the email, add the FSGAN shared directory to your drive:

![Add to drive](https://raw.githubusercontent.com/wiki/YuvalNirkin/fsgan/media/add_to_drive.jpg)




Acknowledgements: We thank Dr. Eyal Gruss, [wangchao0899](https://github.com/wangchao0899), [jjandnn](https://github.com/jjandnn), and [zhuhaozh](https://github.com/zhuhaozh) for helping with this demo.

A Tesla P100 GPU is recommended for this demo. Let's see what we got:

In [0]:
!nvidia-smi

### Installation

In [0]:
import sys
from IPython.display import HTML, clear_output
from base64 import b64encode

# Install the required dependencies
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local
!rm Miniconda3-latest-Linux-x86_64.sh

!conda install pytorch torchvision cudatoolkit=10.1 -c pytorch -y
!pip3 install opencv-python ffmpeg-python youtube-dl yacs

!mkdir -p /content/projects/
%cd /content/projects/
!git clone https://github.com/YuvalNirkin/face_detection_dsfd
!git clone https://github.com/YuvalNirkin/fsgan.git

sys.path += ['/usr/local/lib/python3.7/site-packages', '/content/projects']

# Initialize source and target videos from the repository examples
!mkdir -p /content/data
!cp /content/projects/fsgan/docs/examples/shinzo_abe.mp4 /content/data/source.mp4
!cp /content/projects/fsgan/docs/examples/conan_obrien.mp4 /content/data/target.mp4

# Utility functions
import ffmpeg

def encode_audio(video_path, audio_path, output_path):
  ffmpeg.concat(ffmpeg.input(video_path), ffmpeg.input(audio_path), v=1, a=1) \
    .output(output_path, strict='-2').run(overwrite_output=True)


def display_video(video_path, width=640, clear=True):
  vid_data = open(video_path,'rb').read()
  vid_url = 'data:video/mp4;base64,' + b64encode(vid_data).decode()

  if clear:
    clear_output()

  return HTML(f"""
  <video width={width} controls>
    <source src={vid_url} type="video/mp4">
  </video>
  """)

Mount your Google Drive using the following step or click on "Mount Drive" in the menu to the left

In [0]:
from google.colab import drive
drive.mount('/content/drive')

### Initialize face swapping

In [0]:
import os
from fsgan.inference.swap import FaceSwapping
from fsgan.criterions.vgg_loss import VGGLoss

#@markdown This step should only be done once unless one of the
#@markdown following parameters is changed:

#@markdown ---
#@markdown Path to the weights directory (make sure it is correct):
weights_dir = '/content/drive/My Drive/fsgan/weights' #@param {type:"string"}
#@markdown Number of finetune iterations on the source subject:
finetune_iterations = 800 #@param {type:"slider", min:100, max:2000, step:1}
#@markdown If True, the inner part of the mouth will be removed from the segmentation:
seg_remove_mouth = True #@param {type:"boolean"}
#@markdown Segmentation batch size
seg_batch_size = 24 #@param {type:"slider", min:1, max:64, step:1}
#@markdown Inference batch size
batch_size = 8 #@param {type:"slider", min:1, max:64, step:1}
#@markdown ---


detection_model = os.path.join(weights_dir, 'v2/WIDERFace_DSFD_RES152.pth')
pose_model = os.path.join(weights_dir, 'shared/hopenet_robust_alpha1.pth')
lms_model = os.path.join(weights_dir, 'v2/hr18_wflw_landmarks.pth')
seg_model = os.path.join(weights_dir, 'v2/celeba_unet_256_1_2_segmentation_v2.pth')
reenactment_model = os.path.join(weights_dir, 'v2/nfv_msrunet_256_1_2_reenactment_v2.1.pth')
completion_model = os.path.join(weights_dir, 'v2/ijbc_msrunet_256_1_2_inpainting_v2.pth')
blending_model = os.path.join(weights_dir, 'v2/ijbc_msrunet_256_1_2_blending_v2.pth')
criterion_id_path = os.path.join(weights_dir, 'v2/vggface2_vgg19_256_1_2_id.pth')
criterion_id = VGGLoss(criterion_id_path)


face_swapping = FaceSwapping(
    detection_model=detection_model, pose_model=pose_model, lms_model=lms_model,
    seg_model=seg_model, reenactment_model=reenactment_model,
    completion_model=completion_model, blending_model=blending_model,
    criterion_id=criterion_id,
    finetune=True, finetune_save=True, finetune_iterations=finetune_iterations,
    seg_remove_mouth=finetune_iterations, batch_size=batch_size,
    seg_batch_size=seg_batch_size, encoder_codec='mp4v')

### Optional: Download source video from YouTube
You can change the source URL, start and end times [hh:mm:ss].  
Alternatively, manually upload an image or video to **_data_** (in the menu to the left) and rename to **_source.jpg_** or **_source.mp4_** (click "Refresh" if missing). By default the source video will be taken from the repository examples.

In [0]:
source_url = 'https://www.youtube.com/watch?v=5-s3ANu4eMs' #@param {type:"string"}
source_start = '00:01:40' #@param {type:"string"}
source_end = '00:01:50' #@param {type:"string"}

!mkdir -p /content/data
!rm -dr /content/data/source*
!youtube-dl $source_url --merge-output-format mp4 -o /content/data/source_tmp.mp4
!ffmpeg -y -i /content/data/source_tmp.mp4 -ss $source_start -to $source_end -r 25 /content/data/source.mp4
!rm /content/data/source_tmp.mp4

display_video('/content/data/source.mp4')

### Optional: Download target video from YouTube
You can change the target URL, start and end times [hh:mm:ss].  
Alternatively, manually upload a video to **_data_** (in the menu to the left) and rename to **_target.mp4_** (click "Refresh" if missing). By default the source video will be taken from the repository examples.

In [0]:
target_url = 'https://www.youtube.com/watch?v=YLWqqDVWsXo' #@param {type:"string"}
target_start = '00:01:03' #@param {type:"string"}
target_end = '00:01:09' #@param {type:"string"}

!mkdir -p /content/data
!rm -dr /content/data/target*
!youtube-dl $target_url --merge-output-format mp4 -o /content/data/target_tmp.mp4
!ffmpeg -i /content/data/target_tmp.mp4 -ss $target_start -to $target_end -r 25 /content/data/target.mp4
!rm /content/data/target_tmp.mp4

display_video('/content/data/target.mp4')

### Do face swapping

In [0]:
# Do face swapping
#@markdown ---
#@markdown Toggle whether to finetune the reenactment generator:
finetune = True #@param {type:"boolean"}
#@markdown Source path
source_path = '/content/data/source.mp4' #@param {type:"string"}
#@markdown Source selection method ["longest" | sequence number]:
select_source = 'longest' #@param {type:"string"}
#@markdown Target path
target_path = '/content/data/target.mp4' #@param {type:"string"}
#@markdown Target selection method ["longest" | sequence number]:
select_target = 'longest' #@param {type:"string"}
#@markdown ---
output_tmp_path = '/content/data/output_tmp.mp4'
output_path = '/content/output.mp4'
face_swapping(source_path, target_path, output_tmp_path,
              select_source, select_target, finetune)

# Encode with audio and display result
encode_audio(output_tmp_path, target_path, output_path)
os.remove(output_tmp_path)
display_video(output_path)